In [9]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
%matplotlib inline

torch.manual_seed(1)

import warnings
warnings.filterwarnings('ignore')

Урезанный dataset

In [3]:
df = pd.read_csv('datasets/nyt-ingredients-head.csv')
df.head(5)

,index,input,name,qty,range_end,unit,comment
0,0,1 1/4 cups cooked and pureed fresh butternut s...,butternut squash,1.25,0.0,cup,"cooked and pureed fresh, or 1 10-ounce package..."
1,1,1 cup peeled and cooked fresh chestnuts (about...,chestnuts,1.00,0.0,cup,"peeled and cooked fresh (about 20), or 1 cup c..."
2,2,"1 medium-size onion, peeled and chopped",onion,1.00,0.0,NaN,"medium-size, peeled and chopped"
3,3,"2 stalks celery, chopped coarse",celery,2.00,0.0,stalk,chopped coarse
4,4,1 1/2 tablespoons vegetable oil,vegetable oil,1.50,0.0,tablespoon,NaN


In [4]:
datafile = 'datasets/BIO_recipe_dataset.txt'
lines    = open(datafile, encoding='utf-8').read().strip().split('\n')

In [28]:
# написанная библиотека
# from recipe_utils import form_vocabulary_and_tagset, prepare_data, show_markup, Converter, tag_statistics, plot_confusion_matrix, recipe_statistics, plot_recipe_statistics
import recipe_utils

In [30]:
# словарь и тэги
vocabulary,labels = form_vocabulary_and_tagset(lines)

# рецепты
recipes_w_tags = prepare_data(lines)

len(recipes_w_tags)

49847

In [31]:
# ner для первого рецепта
test_recipe, test_tags = recipes_w_tags[0]
show_markup(test_recipe, test_tags)

In [32]:
converter = Converter(vocabulary,labels)

# проверка работы энкодера и декодера
test_recipe, test_tags = recipes_w_tags[0]


encoded_recipe = converter.words_to_index(test_recipe)
encoded_tags   = converter.tags_to_index(test_tags)

print(encoded_recipe)
print(encoded_tags)
print()


decoded_recipe = converter.indices_to_words(encoded_recipe)
decoded_tags   = converter.indices_to_tags(encoded_tags)

show_markup(decoded_recipe, decoded_tags)

tensor([  57, 2245, 2133, 1520, 4060, 2650, 1846, 4624,   25, 3729,   42,  125,
        3771, 2668, 4624,   25, 2303])
tensor([3, 5, 0, 6, 6, 6, 2, 7, 9, 6, 6, 6, 6, 6, 2, 9, 6])



In [20]:
# разделение на тестовую и тренировочную выборки
# возможно нужно будет попробовать другой способ
training_data = recipes_w_tags[     :40000]
test_data     = recipes_w_tags[40000:]